In [ ]:
# neu co gpu roi thi k can [cpu]
%pip install "audio-separator[cpu]"
%pip install ffmpeg

In [ ]:
%pip install librosa matplotlib numpy

In [ ]:
from audio_separator.separator import Separator

separator = Separator()

separator.load_model('model_bs_roformer_ep_317_sdr_12.9755.ckpt')

In [ ]:
AUDIO_PATH='D:/sound_project/media_files/separated_audio/train/Tobenai Tsubasa - Lily Chou-Chou.mp3'
OUTPUT_PATH = 'D:/sound_project/media_files/output/test/chou-chou'

OUTPUT_NAMES={"Vocals": "vocal_output", "Instrumentals": "instrumental_output"}
output_files = separator.separate(AUDIO_PATH)
print(f"Separation complete! Output file(s): {' '.join(output_files)}")

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

def visualize_separation_results(original_path, vocal_path, sr=44100):
    """
    Vẽ Predicted Vocal Mask và Separated Vocal Spectrogram.
    """
    print(f"Loading original: {original_path}")
    print(f"Loading vocals: {vocal_path}")
    
    # 1. Load audio
    # Load với thời lượng giới hạn (ví dụ 30s) để vẽ cho nhanh và dễ nhìn như ảnh mẫu
    # duration=30: chỉ lấy 30 giây đầu. Bỏ tham số này nếu muốn vẽ toàn bài.
    y_mix, _ = librosa.load(original_path, sr=sr, duration=30, mono=True)
    y_voc, _ = librosa.load(vocal_path, sr=sr, duration=30, mono=True)
    
    # Đảm bảo độ dài bằng nhau (do sai số làm tròn khi load)
    min_len = min(len(y_mix), len(y_voc))
    y_mix = y_mix[:min_len]
    y_voc = y_voc[:min_len]

    # 2. Compute STFT (Short-Time Fourier Transform)
    n_fft = 2048
    hop_length = 512
    
    # Tính Magnitude (biên độ)
    S_mix = np.abs(librosa.stft(y_mix, n_fft=n_fft, hop_length=hop_length))
    S_voc = np.abs(librosa.stft(y_voc, n_fft=n_fft, hop_length=hop_length))

    # 3. Tính toán Mask và Spectrogram
    # Mask ước tính = Tỷ lệ năng lượng của Vocal trên tổng thể
    # Cộng thêm 1e-8 (epsilon) để tránh chia cho 0
    estimated_mask = S_voc / (S_mix + 1e-8)
    estimated_mask = np.clip(estimated_mask, 0, 1) # Giới hạn giá trị từ 0 đến 1
    
    # Spectrogram (chuyển sang thang dB để hiển thị giống mắt người nhìn)
    S_voc_db = librosa.amplitude_to_db(S_voc, ref=np.max)

    # 4. Vẽ hình
    fig, ax = plt.subplots(1, 2, figsize=(15, 6))

    # Hình 1: Predicted Vocal Mask (Ước tính)
    img1 = librosa.display.specshow(estimated_mask, sr=sr, hop_length=hop_length, 
                                    x_axis='time', y_axis='hz', ax=ax[0], cmap='magma')
    ax[0].set_title('Estimated Vocal Mask (Reconstructed)')
    fig.colorbar(img1, ax=ax[0], format="%+2.0f")
    ax[0].set_ylim(0, sr/2) # Hiển thị full dải tần

    # Hình 2: Separated Vocal Spectrogram
    img2 = librosa.display.specshow(S_voc_db, sr=sr, hop_length=hop_length, 
                                    x_axis='time', y_axis='hz', ax=ax[1], cmap='magma')
    ax[1].set_title('Separated Vocal Spectrogram')
    fig.colorbar(img2, ax=ax[1], format="%+2.0f dB")
    ax[1].set_ylim(0, sr/2)

    plt.tight_layout()
    plt.show()

# --- HƯỚNG DẪN SỬ DỤNG ---
# Thay thế đường dẫn bên dưới bằng đường dẫn thực tế của bạn
# input_audio = AUDIO_PATH (biến bạn đã khai báo ở trên)
# output_vocal = đường dẫn file vocal vừa tách ra (xem trong output_files)

# Ví dụ (dựa trên code của bạn):
# visualize_separation_results(AUDIO_PATH, output_files[0]) # Giả sử file đầu tiên là vocal

In [ ]:
# Giả sử output_files trả về danh sách đầy đủ đường dẫn
# Nếu chỉ trả về tên file, bạn cần nối thêm đường dẫn thư mục output
vocal_file_path = output_files[0] # Kiểm tra kỹ xem phần tử nào là vocal

visualize_separation_results(AUDIO_PATH, vocal_file_path)